In [27]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn import Transformer
import torch.nn.functional as F
from torch import Tensor
from torch import nn
import warnings
import random
import torch
import math
import yaml
import json
import os
# warnings.filterwarnings("ignore")

In [28]:
class index2char():
    def __init__(self, root, tokenizer=None):
        if tokenizer is None:
            with open(root + '/tokenizer.yaml', 'r') as f:
                self.tokenizer = yaml.load(f, Loader=yaml.CLoader)
        else:
            self.tokenizer = tokenizer
    
    def __call__(self, indices:list, without_token=True):
        if type(indices) == Tensor:
            indices = indices.tolist()
        result = ''.join([self.tokenizer['index_2_char'][i] for i in indices])
        if without_token:
            result = result.split('[eos]')[0]
            result = result.replace('[sos]', '').replace('[eos]', '').replace('[pad]', '')
        return result

In [29]:
def metrics(pred:list, target:list) -> float:
    """
    pred: list of strings
    target: list of strings

    return: accuracy(%)
    """
    if len(pred) != len(target):
        raise ValueError('length of pred and target must be the same')
    correct = 0
    for i in range(len(pred)):
        if pred[i] == target[i]:
            correct += 1
    return correct / len(pred) * 100

In [30]:
embedding_num = 31
embedding_dim = 512
num_layers = 8
num_heads = 8
ff_dim = 1024
dropout = 0.1

In [31]:
import torch
from torch.utils.data import Dataset
import json
import yaml


class SpellCorrectionDataset(Dataset):
    def __init__(self, root, split:str = 'train', tokenizer=None, padding:int =0):
        super(SpellCorrectionDataset, self).__init__()
        #load your data here
        self.padding = padding
        
        if tokenizer:
            self.tokenizer = tokenizer.tokenizer
        else:
            with open(os.path.join(root, 'tokenizer.yaml'), 'r') as f:
                self.tokenizer = yaml.safe_load(f)
        
        data_path = os.path.join(root, f'{split}.json')
        with open(data_path, 'r') as f:
            self.all_data = json.load(f)
        self.data =[]
        for line in range(len(self.all_data)):
            for input in self.all_data[line]['input']:
                self.data.append({'input':input,'target':self.all_data[line]['target']})
                
        # Add this line right after loading the tokenizer
        # print(f"Tokenizer structure: {self.tokenizer}")

        # Ensure 'char_2_index' is part of the tokenizer dictionary and is a dictionary itself
        assert isinstance(self.tokenizer, dict), "Tokenizer should be a dictionary."
        assert 'char_2_index' in self.tokenizer, "'char_2_index' not found in tokenizer."
        assert isinstance(self.tokenizer['char_2_index'], dict), "'char_2_index' should be a dictionary."

    
    def tokenize(self, text:str):
        # tokenize your text here
        # ex: "data" -> [4, 1, 20, 1]
        
        # 將文本轉換為索引序列
        tokens = [27]
        for char in text:
            tokens.append(self.tokenizer['char_2_index'].get(char))


        #在文本的最後面增加[eos]表示結尾
        tokens.append(28)
        # print(f"tokens:{tokens}")


        # 根據指定的padding進行填充或截斷
        if self.padding > 0:
            tokens = tokens[:self.padding] + [0] * max(0, self.padding - len(tokens))  # 使用 0 進行填充
        return tokens
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # get your data by index here
        # ex: return input_ids, target_ids
        # return type: torch.tensor
        item = self.data[index]
        input_text = item['input']
        input_ids = self.tokenize(input_text)
        target_text = item['target']
        target_ids = self.tokenize(target_text)
        # print(f"input_ids={input_ids}")
        # print(f"input_text={input_text}")
        # print(f"target_text={target_text}")
        # print(f"target_ids={target_ids}")
        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long)

In [32]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, hid_dim, dropout, max_length, batch_first=True):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)

        position = torch.arange(0, max_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hid_dim, 2) * (-torch.log(torch.tensor(10000.0)) / hid_dim))
        
        pos_encoding = torch.zeros(max_length, hid_dim)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        pos_encoding = pos_encoding.unsqueeze(0)
        
        if batch_first:
            pos_encoding = pos_encoding.transpose(0, 1)
        
        self.register_buffer('pos_encoding', pos_encoding)

    def forward(self, x):
        x = x + self.pos_encoding[:x.size(0), :]
        return self.dropout(x)

class Encoder(nn.Module):
    def __init__(self, num_emb, hid_dim, n_layers, n_heads, ff_dim, dropout, max_length=100):
        super(Encoder, self).__init__()
        self.tok_embedding = nn.Embedding(num_emb, hid_dim)
        self.pos_embedding = PositionalEncoding(hid_dim, dropout, max_length, batch_first=True)
        
        # Define Transformer Encoder Layer and Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=hid_dim, nhead=n_heads, dim_feedforward=ff_dim, dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # Apply embeddings and positional encoding
        src = self.tok_embedding(src) * torch.sqrt(torch.tensor(self.tok_embedding.embedding_dim, dtype=torch.float32))
        src = self.pos_embedding(src)
        
        # Pass through the Transformer Encoder
        src = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        return src

class Decoder(nn.Module):
    def __init__(self, num_emb, hid_dim, n_layers, n_heads, ff_dim, dropout, max_length=100):
        super(Decoder, self).__init__()
        self.tok_embedding = nn.Embedding(num_emb, hid_dim)
        self.pos_embedding = PositionalEncoding(hid_dim, dropout, max_length, batch_first=True)
        
        # Define Transformer Decoder Layer and Decoder
        decoder_layer = nn.TransformerDecoderLayer(d_model=hid_dim, nhead=n_heads, dim_feedforward=ff_dim, dropout=dropout, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=n_layers)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        # Apply embeddings and positional encoding
        tgt = self.tok_embedding(tgt) * 0.1  # Apply scaling factor
        tgt = self.pos_embedding(tgt)
        # print("tgt after embedding:", tgt)  # Debugging line
        
        # Pass through the Transformer Decoder
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask, 
                              tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask)
        # print("Decoder output after layer:", output)  # Debugging line
        return output

class TransformerAutoEncoder(nn.Module):
    def __init__(self, num_emb, hid_dim, n_layers, n_heads, ff_dim, dropout, max_length=100, encoder=None):
        super(TransformerAutoEncoder, self).__init__()
        # Initialize encoder and decoder
        self.encoder = encoder if encoder else Encoder(num_emb, hid_dim, n_layers, n_heads, ff_dim, dropout, max_length)
        self.decoder = Decoder(num_emb, hid_dim, n_layers, n_heads, ff_dim, dropout, max_length)
        
        # Fully connected layer to map decoder output to the range of 0-28 (vocab size)
        self.fc_out = nn.Linear(hid_dim, num_emb)

    def forward(self, src, tgt, src_pad_mask=None, tgt_mask=None, tgt_pad_mask=None):
        # Encode the source sequence
        enc_src = self.encoder(src, src_key_padding_mask=src_pad_mask)
        # print("enc_src:", enc_src)  # Check if this output is NaN
        
        # Decode the target sequence using the encoder output as memory
        out = self.decoder(tgt, enc_src, tgt_mask=tgt_mask, memory_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        # print("decoder out:", out)  # Check if this output is NaN
        
        # Pass the decoder output through the fully connected layer to map to the output range
        out = self.fc_out(out)
        
        return out




In [33]:
def gen_padding_mask(src, pad_idx):
    pad_mask = (src == pad_idx)
    # print(pad_mask)
    return pad_mask

def gen_mask(seq):
    seq_length = seq.size(1)
    # Create a mask with `-inf` in the upper triangle and 0s elsewhere
    mask = torch.triu(torch.ones((seq_length, seq_length)), diagonal=1).bool()
    return mask

def get_index(pred, dim=2):
    return pred.clone().argmax(dim=dim)

def random_change_idx(data: torch.Tensor, prob: float = 0.2):
    # randomly change the index of the input data
    return #sample

def random_masked(data: torch.Tensor, prob: float = 0.2, mask_idx: int = 3):
    # randomly mask the input data
    return #sample

# Pretrained encoder with random mask

In [34]:
# You can try to pretrain the Encoder here!

# Train our spelling correction transformer

In [35]:
from tqdm import tqdm
i2c = index2char('./data/')

trainset = SpellCorrectionDataset('./data/', tokenizer=i2c, padding=22)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testset = SpellCorrectionDataset('./data/', split='new_test', tokenizer=i2c, padding=22)
testloader = DataLoader(testset, batch_size=32, shuffle=False)
valset = SpellCorrectionDataset('./data/', split='test', tokenizer=i2c, padding=22)
valloader = DataLoader(valset, batch_size=32, shuffle=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

padding_token = 0  # Example index for padding
sos_token = 1      # Example index for start-of-sequence
ce_loss = torch.nn.CrossEntropyLoss(ignore_index=padding_token)


In [36]:
def decode_indices(indices, i2c_func):
    decoded = []
    for idx in indices:
        idx = idx.item()  # Convert tensor to integer
        try:
            decoded.append(i2c_func([idx]))  # Attempt to decode the index
        except KeyError:
            decoded.append("[UNK]")  # Add unknown token placeholder if index is invalid
    return decoded

In [37]:
def validation(dataloader, model, device, logout=False):
    pred_str_list = []
    tgt_str_list = []
    input_str_list = []
    losses = []
    for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            tgt_input = torch.full_like(tgt, fill_value=0).to(device)  # Fill with padding token (e.g., 0)
            # print(tgt_input.shape)
            tgt_input[:, 0] = 27  # Replace the first token with the <sos> token index (assuming <sos> index is 27)
            for i in range(tgt.shape[1]-1):
                src_pad_mask = gen_padding_mask(src=src, pad_idx=0).to(device)
                tgt_pad_mask = gen_padding_mask(src=tgt, pad_idx=0).to(device)
                tgt_mask = gen_mask(tgt).to(device)
                pred = model(src, tgt_input, src_pad_mask, tgt_mask, tgt_pad_mask)
                # pred = <get the prediction idx from the model>
                # print(pred)
                # assign the prediction idx to the next token of tgt_input
                next_token = get_index(pred=pred)

                # Assign the predicted index to the next position in `tgt_input`
                tgt_input[:, i+1] = next_token[:,i]
                # print(tgt_input)
            for i in range (tgt.shape[0]-1):
                pred_str_list.append(i2c(tgt_input[i].tolist()))
                tgt_str_list.append(i2c(tgt[i].tolist()))
                input_str_list.append(i2c(src[i].tolist()))
                if logout:
                    print('='*30)
                    print(f'input: {input_str_list[-1]}')
                    print(f'pred: {pred_str_list[-1]}')
                    print(f'target: {tgt_str_list[-1]}')
            loss = ce_loss(pred[:, :-1, :].permute(0, 2, 1), tgt[:, 1:])
            losses.append(loss.item())
    print(f"test_acc: {metrics(pred_str_list, tgt_str_list):.2f}", f"test_loss: {sum(losses)/len(losses):.2f}", end=' | ')
    print(f"[pred: {pred_str_list[0]} target: {tgt_str_list[0]}]")

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define model parameters
embedding_num = 31  # vocab size or number of tokens
embedding_dim = 512  # input embedding dimension
num_layers = 6  # number of encoder/decoder layers
num_heads = 8  # number of attention heads
ff_dim = 2048  # feedforward layer dimension
dropout = 0.1  # dropout rate

# Define model components
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_layer = nn.Embedding(embedding_num, embedding_dim).to(device)

# Initialize your custom Transformer model
model = TransformerAutoEncoder(
    num_emb=embedding_num,
    hid_dim=embedding_dim,
    n_layers=num_layers,
    n_heads=num_heads,
    ff_dim=ff_dim,
    dropout=dropout,
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
ce_loss = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)


In [39]:
# encoder.pretrained_mode = False
for eps in range(100):
    # train
    losses = []
    model.train()
    i_bar = tqdm(trainloader, unit='iter', desc=f'epoch{eps}')
    for src, tgt in i_bar:
        src, tgt = src.to(device), tgt.to(device)
        # generate the mask and padding mask
        src_pad_mask = gen_padding_mask(src=src, pad_idx=0).to(device)
        tgt_pad_mask = gen_padding_mask(src=tgt, pad_idx=0).to(device)
        tgt_mask = gen_mask(tgt).to(device)
        optimizer.zero_grad()
        pred = model(src, tgt, src_pad_mask, tgt_mask, tgt_pad_mask).to(device) #[32,22,512]
        loss = ce_loss(pred[:, :-1, :].permute(0, 2, 1), tgt[:, 1:])
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        i_bar.set_postfix_str(f"loss: {sum(losses)/len(losses):.3f}")
    # test
    model.eval()
    with torch.no_grad():
        validation(testloader, model, device)
    model.eval()
    with torch.no_grad():
        validation(valloader, model, device)
    # eval
scheduler.step()  # Ensure `scheduler` is initialized and connected to `optimizer`


epoch0: 100%|██████████| 404/404 [00:12<00:00, 32.65iter/s, loss: 1.062]


test_acc: 0.00 test_loss: 1.75 | [pred: parare target: appreciate]
test_acc: 0.00 test_loss: 1.57 | [pred: dontentent target: contented]


epoch1: 100%|██████████| 404/404 [00:12<00:00, 32.78iter/s, loss: 0.665]


test_acc: 0.00 test_loss: 1.37 | [pred: parectice target: appreciate]
test_acc: 2.08 test_loss: 1.77 | [pred: pontetent target: contented]


epoch2: 100%|██████████| 404/404 [00:12<00:00, 32.41iter/s, loss: 0.565]


...

test_acc: 45.83 test_loss: 3.61 | [pred: appreciate target: appreciate]
test_acc: 77.08 test_loss: 1.28 | [pred: parendent target: contented]


epoch98: 100%|██████████| 404/404 [00:12<00:00, 32.43iter/s, loss: 0.042]


test_acc: 47.92 test_loss: 3.58 | [pred: appreciate target: appreciate]
test_acc: 79.17 test_loss: 0.77 | [pred: contempted target: contented]


epoch99: 100%|██████████| 404/404 [00:12<00:00, 32.03iter/s, loss: 0.048]


test_acc: 39.58 test_loss: 3.79 | [pred: appreciate target: appreciate]
test_acc: 85.42 test_loss: 0.84 | [pred: incompetent target: contented]


In [40]:
validation(testloader, model, device)


test_acc: 39.58 test_loss: 3.79 | [pred: appreciate target: appreciate]
